In [ ]:
from astroquery.vizier import Vizier
vizier = Vizier()

from astropy.coordinates import SkyCoord
import astropy.units as u

import matplotlib.pyplot as plt
import numpy as np
import numpy.ma as ma
import pandas as pd

## Query the LMC variable dataset
Directly get accesss from Vizier with the catalogue ID: `J/A+A/430/603/variables`

In [ ]:
vizier = vizier(columns=["RAJ2000", "DEJ2000", "VType", "Per","<Bmag>", "<Vmag>", "<Imag>","Notes"])
vizier.ROW_LIMIT = 9999

In [ ]:
LMC_rr = vizier.get_catalogs("J/A+A/430/603/variables")

### Select the RR Lyrae stars in type **ab**

In [ ]:
def select_type(catalog, rrtype):
    rrlist = []
    for i in range(len(catalog)):
        if catalog[i]["VType"] == rrtype and catalog[i]["Notes"] != "Incomplete" and catalog[i]["Notes"] != "Blend":

            # Convert the coordinate string to astropy Skycoord
            obs_coord = SkyCoord(
                                 ra=catalog[i]["RAJ2000"], dec=catalog[i]["DEJ2000"], 
                                 unit=(u.hourangle, u.deg), 
                                 frame='icrs'
                                )

            rrlist.append({
                'Coordinate' : obs_coord,
                         'P' : catalog[i]["Per"],
                     'obs_B' : catalog[i]["__Bmag_"],
                     'obs_V' : catalog[i]["__Vmag_"],
                     'obs_I' : catalog[i]["__Imag_"],
            })
    return rrlist

In [ ]:
LMC_rr0 = select_type(LMC_rr[0], "ab")

RR0 samples: 94

In [ ]:
LMC_rr1 = select_type(LMC_rr[0], "c")

RR1 samples: 54

## Extinction
From [Gorski et al. 2020](https://ui.adsabs.harvard.edu/abs/2020ApJ...889..179G/abstract) derived the Magellanic clouds [extinction maps](https://araucaria.camk.edu.pl/index.php/magellanic-clouds-extinction-maps/) with different spatial resolution of map (using 3 arcmin resolute)
- V band:  
    Utilize the relationship $ A_V = R_V \times E(B-V) $. The authors use $ R_V = 3.1 $ ([Cardelli et al. 1989](https://ui.adsabs.harvard.edu/abs/1989ApJ...345..245C/abstract) extinction law)
- B band:  
    Use the definition of $E(B-V)$, which is $E(B-V) = A_B - A_V$
    
- I band:  
    From the extinction law in Cardelli et al., the ratio $A_I/A_V\approx 0.479$

To sum up, the extinction law for each band is:

$$\begin{aligned}
    A_B &= E(B-V) + A_V \\
    A_V &= 3.1 \cdot E(B-V) \\
    A_I &= 0.479 \cdot A_V
\end{aligned}$$

1. Read the reddening map
2. Match observational targets to the map:  
    - Use the angular distance formula to find the closest grid point in the reddening map:  
    $$
        d = \sqrt{(\text{RA}_\text{target} - \text{RA}_\text{grid})^2 + (\text{Dec}_\text{target} - \text{Dec}_\text{grid})^2}
    $$
    Select the grid point with smallest distance $d$
     - Using the [`Skycoord.match_to_catalog_sky`](https://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html)
     
### Error propagation
#### Extinction $A_\lambda$
$$\begin{aligned}
    \sigma_{A_B} &= \sqrt{\sigma_{E(B-V)}^2 + (3.1\sigma_{E(B-V)})^2} \\
    \sigma_{A_V} &= 3.1 \cdot \sigma_{E(B-V)} \\
    \sigma_{A_I} &= 0.479 \cdot \sigma_{E(B-V)}
\end{aligned}$$
#### Corrected magnitude
$$\begin{aligned}
    \sigma_{B_\text{corr}} &= \sigma_{A_B} \\
    \sigma_{V_\text{corr}} &= \sigma_{A_V} \\
    \sigma_{I_\text{corr}} &= \sigma_{A_I}
\end{aligned}$$

In [ ]:
# read the downloaded reddening E(B-V) map
reddening_map = pd.read_csv(
                            "EBV_res3_lmc.txt", 
                            delim_whitespace=True, 
                            comment='#',  # Ignore comment lines
                            names=['RA', 'Dec', 'E_BV', 'Error', 'Sigma_RC', 'N_RC']
                           )

map_coords = SkyCoord(ra=reddening_map["RA"] * u.deg, dec=reddening_map['Dec'] * u.deg)

In [ ]:
R_V = 3.1
AI_AV_ratio = 0.479

In [ ]:
def extinction(EBV_map, mapcoord, rrlist):
    
    corrected =[]
    for data in rrlist:

        # use SkyCoord.match_to_catalog_sky 
        idx, d2d, d3d = data['Coordinate'].match_to_catalog_sky(mapcoord)

        # idx is the index to the catalog (reddening map)
        E_BV = EBV_map['E_BV'][idx]

        A_V = R_V * E_BV
        A_B = A_V + E_BV
        A_I = AI_AV_ratio * A_V

        error_E_BV = EBV_map['Error'][idx]

        B_error = np.sqrt( error_E_BV**2 + (3.1 * error_E_BV)**2 )
        V_error = 3.1 * error_E_BV
        I_error = 0.479 * error_E_BV

        B_corrected = data['obs_B'] - A_B
        V_corrected = data['obs_V'] - A_V
        I_corrected = data['obs_I'] - A_I

        corrected.append({
                   'E_BV' : E_BV,
                    'A_B' : A_B,
                    'A_V' : A_V,
                    'A_I' : A_I,
                      'P' : data['P'],
            'B_corrected' : B_corrected,
            'V_corrected' : V_corrected,
            'I_corrected' : I_corrected, 
                'B_error' : B_error,
                'V_error' : V_error,
                'I_error' : I_error,
        })
    return corrected

#### RR0

In [ ]:
rr0_corrected = extinction(reddening_map, map_coords, LMC_rr0)

#### RR1

In [ ]:
rr1_corrected = extinction(reddening_map, map_coords, LMC_rr1)

### Pan-STARRS1 and Johnson conversion
See [Tory et al.](https://ui.adsabs.harvard.edu/abs/2012ApJ...750...99T/abstract) equation (6) and table (6)  
The quadratic transformation gives more accurate result (lower uncertainty):
$$
    y = A_0 + A_1 x + A_2 x^2
$$

|$y$|$x$|$A_0$|$A_1$|$A_2$|$\pm$|
|:-:|:-:|:-:|:-:|:-:|:-:|
|$(g_{P1}-B)$|$(B-V)$|$-0.108$|$-0.845$|$-0.032$|$0.011$|
|$(r_{P1}-V)$|$(B-V)$|$0.082$|$-0.462$|$0.041$|$0.025$|
|$(i_{P1}-I)$|$(B-V)$|$0.341$|$0.154$|$-0.025$|$0.012$|

$$\begin{aligned}
    g_{P1} - B &= -0.108 - 0.845(B-V) - 0.032(B-V)^2 \\
    r_{P1} - V &= 0.082 + 0.462(B-V) + 0.041(B-V)^2 \\
    i_{P1} - I &= 0.341 - 0.154(B-V) - 0.025(B-V)^2 \\
\end{aligned}$$

###  Wesenheit index

From [Ngeow et al. 2022](https://ui.adsabs.harvard.edu/abs/2022AJ....163..239N/abstract), the Wesenheit indices in $gri$ band is defined as
$$\begin{aligned}
    W^{gr} &= r - 2.905 (g - r) \\
    W^{ri} &= r - 4.051 (r - i) \\
    W^{gi} &= g - 2.274 (g - i)
\end{aligned}$$

#### Error propagation

$$\begin{aligned}
    \sigma_g &= \sqrt{\sigma_B^2+\sigma_a^2} \\
    \sigma_r &= \sqrt{\sigma_V^2+\sigma_a^2} \\
    \sigma_i &= \sqrt{\sigma_I^2+\sigma_a^2} \\
    \sigma_{W^{gr}} &= \sqrt{[(-2.905)\sigma_g]^2+[3.905\sigma_r]^2} \\
    \sigma_{W^{ri}} &= \sqrt{[(-3.051)\sigma_r]^2+[4.051\sigma_i]^2} \\
    \sigma_{W^{gi}} &= \sqrt{[(-1.274)\sigma_g]^2+[2.274\sigma_i]^2}
\end{aligned}$$

In [ ]:
uA_g = 0.011
uA_r = 0.025
uA_i = 0.012

In [88]:
def conversion(corrected):
    gri = {'gmag' : [], 'err_g' : [],
           'rmag' : [], 'err_r' : [],
           'imag' : [], 'err_i' : []}
    
    w   = {'wgr' : [], 'err_wgr' : [],
           'wri' : [], 'err_wri' : [],
           'wgi' : [], 'err_wgi' : []}
    
    P   = {'P_g' : [], 'P_Wgr' : [],
           'P_r' : [], 'P_Wri' : [],
           'P_i' : [], 'P_Wgi' : []}
    
    for data in corrected:
        B  = data['B_corrected']
        uB = data['B_error']
        
        V  = data['V_corrected']
        uV = data['V_error']
        
        I  = data['I_corrected']
        uI = data['I_error']
        
        x  = data['B_corrected'] - data['V_corrected']
        
        ux = np.sqrt( uB**2 + uV**2 )
        
        if ma.getmask(B) == False and ma.getmask(V) == False:
        
            # Johnson-Cousins to Pan-STARRS
            gP1 = -0.108 - 0.845 * x - 0.032 * x**2 + B
            rP1 =  0.082 - 0.462 * x + 0.041 * x**2 + V

            err_g = np.sqrt( uB**2 + uA_g**2 )
            err_r = np.sqrt( uV**2 + uA_r**2 )

            # Wesenheit index W^{gr}
            W_gr = rP1 - 2.905*(gP1 - rP1)
            err_Wgr = np.sqrt( ((1 + 2.905)*err_r)**2 + ((- 2.905)*err_g)**2 )

            gri['gmag'].append(gP1)
            gri['err_g'].append(err_g)
            gri['rmag'].append(rP1)
            gri['err_r'].append(err_r)

            w['wgr'].append(W_gr)
            w['err_wgr'].append(err_Wgr)

            P['P_g'].append(data['P'])
            P['P_r'].append(data['P'])
            P['P_Wgr'].append(data['P'])
        
        # convert to i band
        if ma.getmask(B) == False and ma.getmask(V) == False and ma.getmask(I) == False:

            # Johnson-Cousins to Pan-STARRS
            gP1 = -0.108 - 0.845 * x - 0.032 * x**2 + B
            rP1 =  0.082 - 0.462 * x + 0.041 * x**2 + V
            iP1 =  0.341 - 0.154 * x - 0.025 * x**2 + I

            err_i = np.sqrt( uI**2 + uA_i**2)
            err_g = np.sqrt( uB**2 + uA_g**2 )
            err_r = np.sqrt( uV**2 + uA_r**2 )

            # Wesenheit index W^{ri}, W^{gi}
            W_ri = rP1 - 4.051*(rP1 - iP1)
            W_gi = gP1 - 2.274*(gP1 - iP1)

            err_Wri = np.sqrt( ((1 - 4.051)*err_r)**2 + ((4.051)*err_i)**2 )
            err_Wgi = np.sqrt( ((1 - 2.274)*err_g)**2 + ((2.274)*err_i)**2 )

            gri['imag'].append(iP1)
            gri['err_i'].append(err_i)

            w['wri'].append(W_ri)
            w['err_wri'].append(err_Wri)
            w['wgi'].append(W_gi)
            w['err_wgi'].append(err_Wgi)

            P['P_i'].append(data['P'])
            P['P_Wri'].append(data['P'])
            P['P_Wgi'].append(data['P'])
        
    return gri, w, P

### RR0 conversion

In [ ]:
rr0 = conversion(rr0_corrected)

### RR1 conversion

In [ ]:
rr1 = conversion(rr1_corrected)

Sample numbers:  

||RR0|RR1|
|:-|:-:|:-:|
|$g$|74|32|
|$r$|74|32|
|$i$|58|31|
|$W^{gr}$|74|32|
|$W^{ri}$|58|31|
|$W^{gi}$|58|31|

## Distance modulus $\mu = m - M$

### The distance modulus from old work
From [Clementini et al. 2003](https://ui.adsabs.harvard.edu/abs/2003AJ....125.1309C/abstract), the author used 3 method to estimated the absolute magnitude:  

|Method|$M_V$|$\mu$|
|:-|:-:|:-:|
|Baade-Wesselink method|$0.68 \pm 0.15$|$18.38 \pm 0.16$|
|statistical parallaxes|$0.76 \pm 0.13$|$18.30 \pm 0.14$|
|main-sequence fitting of Galactic globular clusters|$0.61 \pm 0.07$|$18.45 \pm 0.09$|

He then reconciled the long- and short-distance scale on a distance modulus $\mu_{LMC} = 18.515 \pm 0.085$  

### The up-to-date LMC distance modulus research

From [Pietrzyński et al. 2019](https://ui.adsabs.harvard.edu/abs/2019Natur.567..200P/abstract), they computed the mean distance modulus to LMC $\langle \mu \rangle = 18.477 ± 0.004\,(\text{statistical}) ± 0.026\,(\text{systemical})$

In [ ]:
muLMC   = 18.477
u_muLMC = np.sqrt(0.004**2 + 0.026**2)

In [ ]:
def AbsoluteMag(mu, err_mu, data):
    gmag  = data[0]['gmag']
    err_g = data[0]['err_g']
    rmag  = data[0]['rmag']
    err_r = data[0]['err_r']
    imag  = data[0]['imag']
    err_i = data[0]['err_i']

    wgr     = data[1]['wgr']
    err_wgr = data[1]['err_wgr']
    wri     = data[1]['wri']
    err_wri = data[1]['err_wri']
    wgi     = data[1]['wgi']
    err_wgi = data[1]['err_wgi']
    
    def DistModulus(m, u):
        Mag = []
        Err = []
        for mag, err in zip(m, u):
            M = mag - mu
            E = np.sqrt(err**2 + err_mu**2)
            Mag.append(M)
            Err.append(E)
        return Mag, Err
    
    M_dict = {'gMag' : [], 'err_g' : [],
              'rMag' : [], 'err_r' : [],
              'iMag' : [], 'err_i' : []}
    
    W_dict = {'Wgr' : [], 'err_Wgr' : [],
              'Wri' : [], 'err_Wri' : [],
              'Wgi' : [], 'err_Wgi' : []}
    
    M_dict['gMag'], M_dict['err_g'] = DistModulus(gmag, err_g)
    M_dict['rMag'], M_dict['err_r'] = DistModulus(rmag, err_r)
    M_dict['iMag'], M_dict['err_i'] = DistModulus(imag, err_i)
    
    W_dict['Wgr'], W_dict['err_Wgr'] = DistModulus(wgr, err_wgr)
    W_dict['Wri'], W_dict['err_Wri'] = DistModulus(wri, err_wri)
    W_dict['Wgi'], W_dict['err_Wgi'] = DistModulus(wgi, err_wgi)
    
    return M_dict, W_dict

### RR0 distance modulus

In [ ]:
RR0_Mag, RR0_W = AbsoluteMag(muLMC, u_muLMC, rr0)

### RR1 distance modulus

In [ ]:
RR1_Mag, RR1_W = AbsoluteMag(muLMC, u_muLMC, rr1)

## Relations
### PL / PLZ
#### [Ngeow et al. 2022](https://ui.adsabs.harvard.edu/abs/2022AJ....163..239N/abstract)
Equation (1):  
$y = a + b \log P + c[\text{Fe}/\text{H}]$  
(PL relation is a special case for $c$ is setted as $0$)

- RR0  
    $$\begin{aligned}
        M_g &=   0.649 − 0.302 \log P + 0.159[\text{Fe}/\text{H}]\\
        M_r &=   0.337 − 1.090 \log P + 0.139[\text{Fe}/\text{H}]\\
        M_i &=   0.243 − 1.432 \log P + 0.144[\text{Fe}/\text{H}]
    \end{aligned}$$

- RRc (RR1)
    $$\begin{aligned}
        M_g &=   0.411 − 0.342 \log P + 0.092[\text{Fe}/\text{H}]\\
        M_r &= − 0.082 − 1.393 \log P + 0.091[\text{Fe}/\text{H}]\\
        M_i &= − 0.205 − 1.706 \log P + 0.077[\text{Fe}/\text{H}]
    \end{aligned}$$

- RR0 + RR1 (combined sample)  
    $ \log P_0 = \log P_1 + 0.127$
    $$\begin{aligned}
        M_g &=   0.801 − 0.032 \log P + 0.190[\text{Fe}/\text{H}]\\
        M_r &=   0.432 − 0.874 \log P + 0.154[\text{Fe}/\text{H}]\\
        M_i &=   0.249 − 1.362 \log P + 0.115[\text{Fe}/\text{H}]
    \end{aligned}$$

#### [Narloch et al. 2024](https://ui.adsabs.harvard.edu/abs/2024A%26A...689A.138N/abstract)
##### PL: $M_\lambda = a (\log P - \log P_0) + b$
How to determine the $\log P_0$: [Zgirski et al. (2023)](https://ui.adsabs.harvard.edu/abs/2023ApJ...951..114Z/abstract)
- RRab (RR0) Set $\log P_0 = -0.25$
    $$\begin{aligned}
        M_g = -2.242 (\log P - \log P_0) + 0.805 \\
        M_r = -2.528 (\log P - \log P_0) + 0.660 \\
        M_i = -2.799 (\log P - \log P_0) + 0.642 \\
    \end{aligned}$$

- RRc (RR1) Set $\log P_0 = -0.45$
    $$\begin{aligned}
        M_g = - 1.984 (\log P - \log P_0) + 0.605 \\
        M_r = - 2.266 (\log P - \log P_0) + 0.557 \\
        M_i = - 2.393 (\log P - \log P_0) + 0.598 \\
    \end{aligned}$$

- RRab + RRc (combined sample) Set $\log P_0 = -0.25$
    $$\begin{aligned}
        M_g = - 1.500 (\log P - \log P_0) + 0.773 \\
        M_r = - 1.929 (\log P - \log P_0) + 0.634 \\
        M_i = - 2.249 (\log P - \log P_0) + 0.619 \\
    \end{aligned}$$

#### PLZ: $M_\lambda = a (\log P - \log P_0) + b + c ([\text{Fe}/\text{H}] - [\text{Fe}/\text{H}]_0)$
$ \log P_0 = -0.25 $ and $ [\text{Fe}/\text{H}]_0 = -1.5 $ dex
- RRab (RR0)
    $$\begin{aligned}
        M_g = - 0.527 (\log P - \log P_0) + 0.794 + 0.264 ([\text{Fe}/\text{H}] - [\text{Fe}/\text{H}]_0) \\
        M_r = - 1.230 (\log P - \log P_0) + 0.651 + 0.205 ([\text{Fe}/\text{H}] - [\text{Fe}/\text{H}]_0) \\
        M_i = - 1.682 (\log P - \log P_0) + 0.635 + 0.174 ([\text{Fe}/\text{H}] - [\text{Fe}/\text{H}]_0) \\
    \end{aligned}$$

- RRab + RRc (combined sample)
    $$\begin{aligned}
        M_g = - 0.284 (\log P - \log P_0) + 0.791 + 0.289 ([\text{Fe}/\text{H}] - [\text{Fe}/\text{H}]_0) \\
        M_r = - 1.017 (\log P - \log P_0) + 0.650 + 0.228 ([\text{Fe}/\text{H}] - [\text{Fe}/\text{H}]_0) \\
        M_i = - 1.469 (\log P - \log P_0) + 0.633 + 0.198 ([\text{Fe}/\text{H}] - [\text{Fe}/\text{H}]_0) \\
    \end{aligned}$$

### PW / PWZ
#### [Ngeow et al. 2022](https://ui.adsabs.harvard.edu/abs/2022AJ....163..239N/abstract)
- RR0 (RRab)  
    $$\begin{aligned}
        W^{gr} &= - 0.644 - 3.324 \log P + 0.052 [\text{Fe}/\text{H}]\\
        W^{ri} &=   0.093 - 2.600 \log P + 0.193 [\text{Fe}/\text{H}]\\
        W^{gi} &= - 0.198 - 2.908 \log P + 0.142 [\text{Fe}/\text{H}]
    \end{aligned}$$

- RR1 (RRc)  
    $$\begin{aligned}
        W^{gr} &= - 1.327 - 3.680 \log P + 0.013 [\text{Fe}/\text{H}]\\
        W^{ri} &= - 0.884 - 3.503 \log P + 0.067 [\text{Fe}/\text{H}]\\
        W^{gi} &= - 1.086 - 3.696 \log P + 0.055 [\text{Fe}/\text{H}]
    \end{aligned}$$

- RR0 + RR1 (combined sample)  
    $ \log P_0 = \log P_1 + 0.127$
    $$\begin{aligned}
        W^{gr} &= - 0.727 - 3.286 \log P + 0.010 [\text{Fe}/\text{H}]\\
        W^{ri} &=   0.010 - 2.756 \log P + 0.149 [\text{Fe}/\text{H}]\\
        W^{gi} &= - 0.288 - 3.066 \log P + 0.101 [\text{Fe}/\text{H}]
    \end{aligned}$$

#### [Narloch et al. 2024](https://ui.adsabs.harvard.edu/abs/2024A%26A...689A.138N/abstract)
##### PW: $W = a (\log P - \log P_0) + b$
How to determine the $\log P_0$: [Zgirski et al. (2023)](https://ui.adsabs.harvard.edu/abs/2023ApJ...951..114Z/abstract)
- RRab (RR0) Set $\log P_0 = -0.25$
    $$\begin{aligned}
        W^{gr} = - 3.357 (\log P - \log P_0) + 0.239 \\
        W^{ri} = - 3.626 (\log P - \log P_0) + 0.587 \\
        W^{gi} = - 3.508 (\log P - \log P_0) + 0.434 \\
    \end{aligned}$$

- RRc (RR1) Set $\log P_0 = -0.45$
    $$\begin{aligned}
        W^{gr} = - 3.086 (\log P - \log P_0) + 0.415 \\
        W^{ri} = - 2.780 (\log P - \log P_0) + 0.723 \\
        W^{gi} = - 2.914 (\log P - \log P_0) + 0.588 \\
    \end{aligned}$$

- RRab + RRc (combined sample) Set $\log P_0 = -0.25$
    $$\begin{aligned}
        W^{gr} = - 3.175 (\log P - \log P_0) + 0.232 \\
        W^{ri} = - 3.226 (\log P - \log P_0) + 0.574 \\
        W^{gi} = - 3.157 (\log P - \log P_0) + 0.424 \\
    \end{aligned}$$

#### PWZ: $W = a (\log P - \log P_0) + b + c ([\text{Fe}/\text{H}] - [\text{Fe}/\text{H}]_0)$
$ \log P_0 = -0.25 $ and $ [\text{Fe}/\text{H}]_0 = -1.5 $ dex
- RRab (RR0)
    $$\begin{aligned}
        W^{gr} = - 3.273 (\log P - \log P_0) + 0.239 + 0.033 ([\text{Fe}/\text{H}] - [\text{Fe}/\text{H}]_0) \\
        W^{ri} = - 3.061 (\log P - \log P_0) + 0.584 + 0.082 ([\text{Fe}/\text{H}] - [\text{Fe}/\text{H}]_0) \\
        W^{gi} = - 3.154 (\log P - \log P_0) + 0.433 + 0.060 ([\text{Fe}/\text{H}] - [\text{Fe}/\text{H}]_0) \\
    \end{aligned}$$

- RRab + RRc (combined sample)
    $$\begin{aligned}
        W^{gr} = - 3.148 (\log P - \log P_0) + 0.238 + 0.050 ([\text{Fe}/\text{H}] - [\text{Fe}/\text{H}]_0) \\
        W^{ri} = - 2.848 (\log P - \log P_0) + 0.583 + 0.105 ([\text{Fe}/\text{H}] - [\text{Fe}/\text{H}]_0) \\
        W^{gi} = - 2.979 (\log P - \log P_0) + 0.432 + 0.081 ([\text{Fe}/\text{H}] - [\text{Fe}/\text{H}]_0) \\
    \end{aligned}$$

###  Metallicity
Using the [Choudhury et al.](https://ui.adsabs.harvard.edu/abs/2016MNRAS.455.1855C/abstract)  
overall metallicity range for the LMC bar is around
$\langle [\text{Fe}/\text{H}] \rangle = -0.34$ dex  

It is much proper to use the average value from [Clementini et al. 2005](https://ui.adsabs.harvard.edu/abs/2003AJ....125.1309C/abstract)
$\langle [\text{Fe}/\text{H}] \rangle = -1.48 \pm 0.03$ dex

In [ ]:
Fe_H = -1.48

In [ ]:
print(Fe_H)

In [ ]:
def PLZ_relations(P_rr0, P_rr1, FeH, Type):
    Ngeow   = {}
    Narloch = {}
    logP    = {}
    P_space = {}
    
    if Type == '0':
        P0 = -0.25

        logP['g'] = np.log10(P_rr0['P_g'])
        logP['r'] = np.log10(P_rr0['P_r'])
        logP['i'] = np.log10(P_rr0['P_i'])
        
        P_space['g'] = np.linspace(min(logP['g']), max(logP['g']), 200)
        P_space['r'] = np.linspace(min(logP['r']), max(logP['r']), 200)
        P_space['i'] = np.linspace(min(logP['i']), max(logP['i']), 200)
        
        Ngeow['Mg'] = 0.649 - 0.302 * P_space['g'] + 0.159 * FeH
        Ngeow['Mr'] = 0.337 - 1.090 * P_space['r'] + 0.139 * FeH
        Ngeow['Mi'] = 0.243 - 1.432 * P_space['i'] + 0.144 * FeH
        
        if FeH == 0:
            Narloch['Mg'] = -2.242 * (P_space['g'] - P0) + 0.805
            Narloch['Mr'] = -2.528 * (P_space['r'] - P0) + 0.660
            Narloch['Mi'] = -2.799 * (P_space['i'] - P0) + 0.642
        else:
            Narloch['Mg'] = - 0.527 * (P_space['g'] - P0) + 0.794 + 0.264 * (FeH - (-1.5))
            Narloch['Mr'] = - 1.230 * (P_space['r'] - P0) + 0.651 + 0.205 * (FeH - (-1.5))
            Narloch['Mi'] = - 1.682 * (P_space['i'] - P0) + 0.635 + 0.174 * (FeH - (-1.5))

    if Type == '1':
        P0 = -0.45
        logP['g'] = np.log10(P_rr1['P_g'])
        logP['r'] = np.log10(P_rr1['P_r'])
        logP['i'] = np.log10(P_rr1['P_i'])
    
        P_space['g'] = np.linspace(min(logP['g']), max(logP['g']), 200)
        P_space['r'] = np.linspace(min(logP['r']), max(logP['r']), 200)
        P_space['i'] = np.linspace(min(logP['i']), max(logP['i']), 200)
        
        Ngeow['Mg'] =   0.411 - 0.342 * P_space['g'] + 0.092 * FeH
        Ngeow['Mr'] = - 0.082 - 1.393 * P_space['r'] + 0.091 * FeH
        Ngeow['Mi'] = - 0.205 - 1.706 * P_space['i'] + 0.077 * FeH
        
        if FeH == 0:
            Narloch['Mg'] = - 1.984 * (P_space['g'] - P0) + 0.605
            Narloch['Mr'] = - 2.266 * (P_space['r'] - P0) + 0.557
            Narloch['Mi'] = - 2.393 * (P_space['i'] - P0) + 0.598
            
        else:
            Narloch['Mg']  = []
            Narloch['Mr']  = []
            Narloch['Mi']  = []
    
    if Type == 'Combined':
        P0 = -0.25
        logP = {'rr0' : {},
                'rr1' : {}}
        logP['rr0']['g'] = np.log10(P_rr0['P_g'])
        logP['rr0']['r'] = np.log10(P_rr0['P_r'])
        logP['rr0']['i'] = np.log10(P_rr0['P_i'])
        
        logP['rr1']['g'] = np.log10(P_rr1['P_g']) + 0.127
        logP['rr1']['r'] = np.log10(P_rr1['P_r']) + 0.127
        logP['rr1']['i'] = np.log10(P_rr1['P_i']) + 0.127
        
        P_space['g'] = np.linspace(min(logP['rr1']['g']), max(logP['rr0']['g']), 200)
        P_space['r'] = np.linspace(min(logP['rr1']['r']), max(logP['rr0']['r']), 200)
        P_space['i'] = np.linspace(min(logP['rr1']['i']), max(logP['rr0']['i']), 200)                                                                                           
        
        Ngeow['Mg'] = 0.801 - 0.032 * P_space['g'] + 0.190 * FeH
        Ngeow['Mr'] = 0.432 - 0.874 * P_space['r'] + 0.154 * FeH
        Ngeow['Mi'] = 0.249 - 1.362 * P_space['i'] + 0.115 * FeH
        
        if FeH == 0:
            Narloch['Mg'] = - 1.500 * (P_space['g'] - P0) + 0.773
            Narloch['Mr'] = - 1.929 * (P_space['r'] - P0) + 0.634
            Narloch['Mi'] = - 2.249 * (P_space['i'] - P0) + 0.619
            
        else:
            Narloch['Mg']  = - 0.284 * (P_space['g'] - P0) + 0.791 + 0.289 * (FeH - (-1.5))
            Narloch['Mr']  = - 1.017 * (P_space['r'] - P0) + 0.650 + 0.228 * (FeH - (-1.5))
            Narloch['Mi']  = - 1.469 * (P_space['i'] - P0) + 0.633 + 0.198 * (FeH - (-1.5))
        
    return Ngeow, Narloch, logP, P_space 

In [ ]:
def PWZ_relations(P_rr0, P_rr1, FeH, Type):
    Ngeow   = {}
    Narloch = {}
    logP    = {}
    P_space = {}
    
    if Type == '0':
        P0 = -0.25

        logP['Wgr'] = np.log10(P_rr0['P_Wgr'])
        logP['Wri'] = np.log10(P_rr0['P_Wri'])
        logP['Wgi'] = np.log10(P_rr0['P_Wgi'])
        
        P_space['Wgr'] = np.linspace(min(logP['Wgr']), max(logP['Wgr']), 200)
        P_space['Wri'] = np.linspace(min(logP['Wri']), max(logP['Wri']), 200)
        P_space['Wgi'] = np.linspace(min(logP['Wgi']), max(logP['Wgi']), 200)
        
        Ngeow['Wgr'] = - 0.644 - 3.324 * P_space['Wgr'] + 0.052 * FeH
        Ngeow['Wri'] =   0.093 - 2.600 * P_space['Wri'] + 0.193 * FeH
        Ngeow['Wgi'] = - 0.198 - 2.908 * P_space['Wgi'] + 0.142 * FeH
        
        if FeH == 0:
            Narloch['Wgr'] = - 3.357 * (P_space['Wgr'] - P0) + 0.239
            Narloch['Wri'] = - 3.626 * (P_space['Wri'] - P0) + 0.587
            Narloch['Wgi'] = - 3.508 * (P_space['Wgi'] - P0) + 0.434
        
        else: 
            Narloch['Wgr']  = - 3.273 * (P_space['Wgr'] - P0) + 0.239 + 0.033 * (FeH - (-1.5))
            Narloch['Wri']  = - 3.061 * (P_space['Wri'] - P0) + 0.584 + 0.082 * (FeH - (-1.5))
            Narloch['Wgi']  = - 3.154 * (P_space['Wgi'] - P0) + 0.433 + 0.060 * (FeH - (-1.5))

    if Type == '1':
        P0 = -0.45
        logP['Wgr'] = np.log10(P_rr1['P_Wgr'])
        logP['Wri'] = np.log10(P_rr1['P_Wri'])
        logP['Wgi'] = np.log10(P_rr1['P_Wgi'])
    
        P_space['Wgr'] = np.linspace(min(logP['Wgr']), max(logP['Wgr']), 200)
        P_space['Wri'] = np.linspace(min(logP['Wri']), max(logP['Wri']), 200)
        P_space['Wgi'] = np.linspace(min(logP['Wgi']), max(logP['Wgi']), 200)
        
        Ngeow['Wgr'] = - 1.327 - 3.680 * P_space['Wgr'] + 0.013 * FeH
        Ngeow['Wri'] = - 0.884 - 3.503 * P_space['Wri'] + 0.067 * FeH
        Ngeow['Wgi'] = - 1.086 - 3.696 * P_space['Wgi'] + 0.055 * FeH
        
        if FeH == 0:
            Narloch['Wgr'] = - 3.086 * (P_space['Wgr'] - P0) + 0.415
            Narloch['Wri'] = - 2.780 * (P_space['Wri'] - P0) + 0.723
            Narloch['Wgi'] = - 2.914 * (P_space['Wgi'] - P0) + 0.588
        
        else: 
            Narloch['Wgr']  = []
            Narloch['Wri']  = []
            Narloch['Wgi']  = []
    
    if Type == 'Combined':
        P0 = -0.25
        logP = {'rr0' : {},
                'rr1' : {}}
        logP['rr0']['Wgr'] = np.log10(P_rr0['P_Wgr'])
        logP['rr0']['Wri'] = np.log10(P_rr0['P_Wri'])
        logP['rr0']['Wgi'] = np.log10(P_rr0['P_Wgi'])
        
        logP['rr1']['Wgr'] = np.log10(P_rr1['P_Wgr']) + 0.127
        logP['rr1']['Wri'] = np.log10(P_rr1['P_Wri']) + 0.127
        logP['rr1']['Wgi'] = np.log10(P_rr1['P_Wgi']) + 0.127
        
        P_space['Wgr'] = np.linspace(min(logP['rr1']['Wgr']), max(logP['rr0']['Wgr']), 200)
        P_space['Wri'] = np.linspace(min(logP['rr1']['Wri']), max(logP['rr0']['Wri']), 200)
        P_space['Wgi'] = np.linspace(min(logP['rr1']['Wgi']), max(logP['rr0']['Wgi']), 200)                                                                                           
        
        Ngeow['Wgr'] =  - 0.727 - 3.286 * P_space['Wgr'] + 0.010 * FeH
        Ngeow['Wri'] =    0.010 - 2.756 * P_space['Wri'] + 0.149 * FeH
        Ngeow['Wgi'] =  - 0.288 - 3.066 * P_space['Wgi'] + 0.101 * FeH
        
        if FeH == 0:
            Narloch['Wgr'] = - 3.175 * (P_space['Wgr'] - P0) + 0.232
            Narloch['Wri'] = - 3.226 * (P_space['Wri'] - P0) + 0.574
            Narloch['Wgi'] = - 3.157 * (P_space['Wgi'] - P0) + 0.424
        
        else: 
            Narloch['Wgr']  = - 3.148 * (P_space['Wgr'] - P0) + 0.238 + 0.050 * (FeH - (-1.5))
            Narloch['Wri']  = - 2.848 * (P_space['Wri'] - P0) + 0.583 + 0.105 * (FeH - (-1.5))
            Narloch['Wgi']  = - 2.979 * (P_space['Wgi'] - P0) + 0.432 + 0.081 * (FeH - (-1.5))
    
    return Ngeow, Narloch, logP, P_space 

In [ ]:
def plot_relation(period_space, PL_data, obs_period, RR0, RR1, Type, FeH, MorW):
    fig, axs = plt.subplots(3, 1, figsize=(8, 12), sharex=True)
    
    def PLZax_setup(ax, period_space, PL_data, obs_period, RR0, RR1, band, Type):
        if band == 'g':
            color0 = 'blue'
            color1 = 'darkorange'
            ylabel = r'$M_g$ [mag]'
        elif band == 'r':
            color0 = 'green'
            color1 = 'deeppink'
            ylabel = r'$M_r$ [mag]'
        elif band == 'i':
            color0 = 'red'
            color1 = 'darkturquoise'
            ylabel = r'$M_i$ [mag]'
        ax.plot(period_space[band], PL_data['Ng'][f'M{band}'], color='k', label='Ngeow et al. (2022)')
        ax.plot(period_space[band], PL_data['Nar'][f'M{band}'], color='k', linestyle='dashed', label='Narloch et al. (2024)')
        if Type == 'RR0':
            ax.scatter(obs_period[band], RR0[f'{band}Mag'], color=color0)
        elif Type == 'RR1':
            ax.scatter(obs_period[band], RR1[f'{band}Mag'], color=color0)
        elif Type == 'Combined':
            ax.scatter(obs_period['rr0'][band], RR0[f'{band}Mag'], color=color0, label='RR0')
            ax.scatter(obs_period['rr1'][band], RR1[f'{band}Mag'], color=color1, label='RR1')
        ax.invert_yaxis()  # Reverse y-axis for magnitude
        ax.set_ylabel(ylabel)
        ax.legend(loc='upper left')
    
    def PWZax_setup(ax, period_space, PL_data, obs_period, RR0, RR1, band, Type):
        if band == 'Wgr':
            color0 = 'blue'
            color1 = 'darkorange'
            ylabel = r'$W^{gr}$'
        elif band == 'Wri':
            color0 = 'green'
            color1 = 'deeppink'
            ylabel = r'$W^{ri}$'
        elif band == 'Wgi':
            color0 = 'red'
            color1 = 'darkturquoise'
            ylabel = r'$W^{gi}$'
        ax.plot(period_space[band], PL_data['Ng'][f'{band}'], color='k', label='Ngeow et al. (2022)')
        ax.plot(period_space[band], PL_data['Nar'][f'{band}'], color='k', linestyle='dashed', label='Narloch et al. (2024)')
        if Type == 'RR0':
            ax.scatter(obs_period[band], RR0[f'{band}'], color=color0)
        elif Type == 'RR1':
            ax.scatter(obs_period[band], RR1[f'{band}'], color=color0)
        elif Type == 'Combined':
            ax.scatter(obs_period['rr0'][band], RR0[f'{band}'], color=color0, label='RR0')
            ax.scatter(obs_period['rr1'][band], RR1[f'{band}'], color=color1, label='RR1')
        ax.invert_yaxis()  # Reverse y-axis for magnitude
        ax.set_ylabel(ylabel)
        ax.legend(loc='upper left')

    if MorW == 'Mag':
        if FeH == True:
            title = f'PLZ Relation for {Type} Sample ([Fe/H]=-1.48)'
        else:
            title = f'PL relation for {Type} Sample'
        PLZax_setup(axs[0], P_space, PLdata, P, RR0, RR1, 'g', Type)
        axs[0].set_title(title)
        PLZax_setup(axs[1], P_space, PLdata, P, RR0, RR1, 'r', Type)
        PLZax_setup(axs[2], P_space, PLdata, P, RR0, RR1, 'i', Type)
    elif MorW == 'W':
        if FeH == True:
            title = f'PWZ Relation for {Type} Sample ([Fe/H]=-1.48)'
        else:
            title = f'PW relation for {Type} Sample'
        PWZax_setup(axs[0], P_space, PLdata, P, RR0, RR1, 'Wgr', Type)
        axs[0].set_title(title)
        PWZax_setup(axs[1], P_space, PLdata, P, RR0, RR1, 'Wri', Type)
        PWZax_setup(axs[2], P_space, PLdata, P, RR0, RR1, 'Wgi', Type)
    
    axs[2].set_xlabel(r'$\log P$ [days]')

## Calculations and Plot
### RR0 PL relation

In [ ]:
Ng, Nar, P, P_space = PLZ_relations(rr0[2], rr1[2], 0, '0')

PLdata = {'Ng':Ng, 'Nar':Nar}

plot_relation(P_space, PLdata, P, RR0_Mag, RR1_Mag, 'RR0', False, 'Mag')
plt.tight_layout()
plt.savefig('LMC-RR0-PL_compare-241209.pdf', dpi=300)
plt.show()

### RR0 PLZ relation

In [ ]:
Ng, Nar, P, P_space = PLZ_relations(rr0[2], rr1[2], Fe_H, '0')

PLdata = {'Ng':Ng, 'Nar':Nar}

plot_relation(P_space, PLdata, P, RR0_Mag, RR1_Mag, 'RR0', True, 'Mag')
plt.tight_layout()
plt.savefig('LMC-RR0-PLZ_compare-241209.pdf', dpi=300)
plt.show()

### RR1 PL relation

In [ ]:
Ng, Nar, P, P_space = PLZ_relations(rr0[2], rr1[2], 0, '1')

PLdata = {'Ng':Ng, 'Nar':Nar}

plot_relation(P_space, PLdata, P, RR0_Mag, RR1_Mag, 'RR1', False, 'Mag')
plt.tight_layout()
plt.savefig('LMC-RR1-PL_compare-241209.pdf', dpi=300)
plt.show()

### Combined Sample PL relation

In [ ]:
Ng, Nar, P, P_space = PLZ_relations(rr0[2], rr1[2], 0, 'Combined')

PLdata = {'Ng':Ng, 'Nar':Nar}

plot_relation(P_space, PLdata, P, RR0_Mag, RR1_Mag, 'Combined', False, 'Mag')
plt.tight_layout()
plt.savefig('LMC-Combined-PL_compare-241209.pdf', dpi=300)
plt.show()

### Combined Sample PLZ relation

In [ ]:
Ng, Nar, P, P_space = PLZ_relations(rr0[2], rr1[2], Fe_H, 'Combined')

PLdata = {'Ng':Ng, 'Nar':Nar}

plot_relation(P_space, PLdata, P, RR0_Mag, RR1_Mag, 'Combined', True, 'Mag')
plt.tight_layout()
plt.savefig('LMC-Combined-PLZ_compare-241209.pdf', dpi=300)
plt.show()

### RR0 PW relation

In [ ]:
Ng, Nar, P, P_space = PWZ_relations(rr0[2], rr1[2], 0, '0')

PLdata = {'Ng':Ng, 'Nar':Nar}

plot_relation(P_space, PLdata, P, RR0_W, RR1_W, 'RR0', False, 'W')
plt.tight_layout()
plt.savefig('LMC-RR0-PW_compare-241209.pdf', dpi=300)
plt.show()

### RR0 PWZ relation

In [ ]:
Ng, Nar, P, P_space = PWZ_relations(rr0[2], rr1[2], Fe_H, '0')

PLdata = {'Ng':Ng, 'Nar':Nar}

plot_relation(P_space, PLdata, P, RR0_W, RR1_W, 'RR0', True, 'W')
plt.tight_layout()
plt.savefig('LMC-RR0-PWZ_compare-241209.pdf', dpi=300)
plt.show()

### RR1 PW relation

In [ ]:
Ng, Nar, P, P_space = PWZ_relations(rr0[2], rr1[2], 0, '1')

PLdata = {'Ng':Ng, 'Nar':Nar}

plot_relation(P_space, PLdata, P, RR0_W, RR1_W, 'RR1', False, 'W')
plt.tight_layout()
plt.savefig('LMC-RR1-PW_compare-241209.pdf', dpi=300)
plt.show()

### Combined Sample PW relation

In [ ]:
Ng, Nar, P, P_space = PWZ_relations(rr0[2], rr1[2], 0, 'Combined')

PLdata = {'Ng':Ng, 'Nar':Nar}

plot_relation(P_space, PLdata, P, RR0_W, RR1_W, 'Combined', False, 'W')
plt.tight_layout()
plt.savefig('LMC-Combined-PW_compare-241209.pdf', dpi=300)
plt.show()

### Combined Sample PWZ relation

In [ ]:
Ng, Nar, P, P_space = PWZ_relations(rr0[2], rr1[2], Fe_H, 'Combined')

PLdata = {'Ng':Ng, 'Nar':Nar}

plot_relation(P_space, PLdata, P, RR0_W, RR1_W, 'Combined', True, 'W')
plt.tight_layout()
plt.savefig('LMC-Combined-PWZ_compare-241209.pdf', dpi=300)
plt.show()